In [1]:
import pandas as pd
import numpy as np
from rdkit import Chem
from tqdm import tqdm
import torch
from sklearn.model_selection import train_test_split
from torch_geometric.data import Data
from rdkit.Chem import Descriptors, rdmolops

from sklearn.model_selection import train_test_split

import pickle
import sys
sys.path.append('../qmdesc')
from qmdesc.handler import ReactivityDescriptorHandler

In [2]:
#have to read the RDKit SMILES for index compatibility

df = pd.read_csv('data/ligand_data_final.csv')
df = df.loc[df['Hemilabile'] == False]

#Convert to Mol objects
from rdkit import RDLogger
RDLogger.DisableLog('rdApp.*')

labels = []
denticities = []
mol_ls = []
for i, x in tqdm(df.iterrows(), total=len(df)):
    try:
        mol = Chem.MolFromSmiles(x['SMILES_without_X_rdkit'])
        label_ls = [int(j) for j in x['Padded_catoms_rdkit'].strip('[]').replace(',', '').split()]
        denticity = x['Ligand_Denticities']
        mol_ls.append(mol)
        labels.append(label_ls)
        denticities.append(denticity)
    except:
        pass
len(mol_ls)

100%|██████████| 72585/72585 [00:16<00:00, 4413.08it/s]


72585

In [4]:
allowed_atoms = {1, 6, 7, 8, 9, 15, 16, 17, 35, 53} #H, C, N, O, F, P, S, Cl, Br, I
filtered_mol_ls = []
filtered_labels = []
filtered_denticities = []
num_failed = 0

#Filter: ensure atoms are those specified above, no radicals (RDKit kekulized correctly)
#Ensure no trivial (1-heavy-atom) ligands, no heavily charged ligands, and ligand size less than 50

for i in tqdm(range(len(mol_ls))):
    #Some of the mol objects fail to generate, skip over those here
    if mol_ls[i] is None:
        num_failed += 1
        continue
    atom_list = [a.GetAtomicNum() for a in mol_ls[i].GetAtoms()]
    if set(atom_list).issubset(allowed_atoms) and \
        Descriptors.NumRadicalElectrons(mol_ls[i]) == 0 and\
        len(atom_list) >= 2 and \
        Chem.GetFormalCharge(mol_ls[i]) <= 2 and Chem.GetFormalCharge(mol_ls[i]) >= -4 and \
        len(atom_list) <= 50:
        filtered_mol_ls.append(mol_ls[i])
        filtered_labels.append(labels[i])
        filtered_denticities.append(denticities[i])
len(filtered_mol_ls), num_failed

100%|██████████| 72585/72585 [00:07<00:00, 10146.84it/s]


(46719, 0)

In [2]:
# with open('data/filtered_labels_smiles.pkl', 'wb') as f:
#     pickle.dump(filtered_labels, f, protocol=pickle.HIGHEST_PROTOCOL)
# with open('data/filtered_mol_ls_smiles.pkl', 'wb') as f:
#     pickle.dump(filtered_mol_ls, f, protocol=pickle.HIGHEST_PROTOCOL)
# with open('data/filtered_denticities_smiles.pkl', 'wb') as f:
#     pickle.dump(filtered_denticities, f, protocol=pickle.HIGHEST_PROTOCOL)
with open('data/filtered_labels_smiles.pkl', 'rb') as f:
    filtered_labels = pickle.load(f)
with open('data/filtered_mol_ls_smiles.pkl', 'rb') as f:
    filtered_mol_ls = pickle.load(f)
with open('data/filtered_denticities_smiles.pkl', 'rb') as f:
    filtered_denticities = pickle.load(f)

In [3]:
def normalize_bw_neg1and1(dictionary: dict):
    values = list(dictionary.values())
    for k, v in dictionary.items():
        dictionary[k] = 2*(v - min(values))/(max(values) - min(values)) - 1
    return dictionary

In [4]:
order_string = {
            Chem.rdchem.BondType.SINGLE: np.array([1, 0, 0, 0]),
            Chem.rdchem.BondType.DOUBLE: np.array([0, 1, 0, 0]),
            Chem.rdchem.BondType.TRIPLE: np.array([0, 0, 1, 0]),
            Chem.rdchem.BondType.AROMATIC: np.array([0, 0, 0, 1]),
}

pt = Chem.GetPeriodicTable()

allowed_atoms = [6, 7, 8, 9, 15, 16, 17, 35, 53] #C, N, O, F, P, S, Cl, Br, I
atoms_one_hot_key = np.eye(len(allowed_atoms))

electronegativities = {
    1: 2.2,
    6: 2.55,
    7: 3.04,
    8: 3.44,
    9: 3.98,
    15: 2.19,
    16: 2.58,
    17: 3.16,
    35: 2.96,
    53: 2.66
}

ionization_energies = {
    1: 13.6,
    6: 11.3,
    7: 14.5,
    8: 13.6,
    9: 17.4,
    15: 10.5,
    16: 10.4,
    17: 13.0,
    35: 11.8,
    53: 10.45
}

valence_electrons = {
    1:  1,
    6:  4,
    7:  5,
    8:  6,
    9:  7,
    15: 5,
    16: 6,
    17: 7,
    35: 7,
    53: 7
}

covalent_radii = {i: pt.GetRcovalent(i) for i in allowed_atoms}
atomic_mass = {i: pt.GetAtomicWeight(i) for i in allowed_atoms}

def normalize_bw_neg1and1(dictionary: dict):
    values = list(dictionary.values())
    for k, v in dictionary.items():
        dictionary[k] = 2*(v - min(values))/(max(values) - min(values)) - 1
    return dictionary

electronegativities = normalize_bw_neg1and1(electronegativities)
ionization_energies = normalize_bw_neg1and1(ionization_energies)
valence_electrons = normalize_bw_neg1and1(valence_electrons)
covalent_radii = normalize_bw_neg1and1(covalent_radii)
atomic_mass = normalize_bw_neg1and1(atomic_mass)

# from QMDesc --> uses ChemProp calculated descriptors
handler = ReactivityDescriptorHandler()

def get_features(ligand, label):
    #Get a list of atomic numbers for each ligand
    atoms_one_hot = np.stack([atoms_one_hot_key[allowed_atoms.index(a.GetAtomicNum())] for a in ligand.GetAtoms()])
    other_atom_features = [[electronegativities[a.GetAtomicNum()],
                            ionization_energies[a.GetAtomicNum()],
                            valence_electrons[a.GetAtomicNum()],
                            covalent_radii[a.GetAtomicNum()],
                            atomic_mass[a.GetAtomicNum()]] for a in ligand.GetAtoms()]

    # QM atom features from QMDesc
    qm_features_dict = handler.predict(Chem.MolToSmiles(ligand))
    qm_atom_features = np.hstack((qm_features_dict['partial_charge'][:,np.newaxis],
                                  qm_features_dict['fukui_neu'][:,np.newaxis],
                                  qm_features_dict['fukui_elec'][:,np.newaxis],
                                  qm_features_dict['NMR'][:,np.newaxis]))
    
    # QM bond features -- don't include unsure if mapping lines up
    # qm_features_dict['bond_order']
    # qm_features_dict['bond_length']
    
    atom_features = np.hstack((atoms_one_hot, np.array(other_atom_features), qm_atom_features))
    
    
    #Get the number of atoms for each ligand
    natoms = len(ligand.GetAtoms())
    
    #Get the adjacency matrix for each ligand
    adj = rdmolops.GetAdjacencyMatrix(ligand)
    
    #Get the edge index list in COO format for each ligand
    edge_index = (adj>0).nonzero()
    
    #Get the edge features (one-hot encoding of the bond type
    feats = np.zeros((4, len(edge_index[0])))
    for i in range(len(edge_index[0])):
        feats[:, i] = order_string[ligand.GetBondBetweenAtoms(int(edge_index[0][i]), int(edge_index[1][i])).GetBondType()]
    edge_feats = feats.T

    #Get the probability of each atom being a connecting atom (0 or 1)
    y = label

    return atom_features, natoms, edge_index, edge_feats, y

atom_features_list = []
natom_list = []
edge_index_list = []
edge_feats_list = []
y_list = []
denticities_list = []
num_failed = 0

for idx in tqdm(range(len(filtered_mol_ls))):
    ligand = filtered_mol_ls[idx]
    label = filtered_labels[idx]
    denticity = filtered_denticities[idx]
    if len(ligand.GetAtoms()) < 2:
        continue
    try:
        atom_features, natoms, edge_index, edge_feats, y = get_features(ligand, label)
        atom_features_list.append(atom_features)
        natom_list.append(natoms)
        edge_index_list.append(edge_index)
        edge_feats_list.append(edge_feats)
        y_list.append(y)
        denticities_list.append(denticity)
    except:
        num_failed += 1

len(atom_features_list), num_failed

100%|██████████| 46719/46719 [04:22<00:00, 177.88it/s]


(46690, 29)

In [30]:
# remove ligands where all atoms coordinate
bad_ids = []
for i, y in enumerate(y_list):
    if all([j==1 for j in y]) or all([j==0 for j in y]):
        bad_ids.append(i)
bad_ids -= np.arange(len(bad_ids)) # adjust for the fact that list indices will decrease as we remove them one by one
for i in bad_ids:
    atom_features_list.pop(i)
    natom_list.pop(i)
    edge_index_list.pop(i)
    edge_feats_list.pop(i)
    y_list.pop(i)
    denticities_list.pop(i)

In [31]:
# Confirm this works
for i, y in enumerate(y_list):
    if all([j==1 for j in y]) or all([j==0 for j in y]):
        print(i)

In [32]:
# train/test
(train_atom_list, test_atom_list,
 train_natom_list, test_natom_list,
 train_edge_index_list, test_edge_index_list,
 train_edge_feats_list, test_edge_feats_list,
 train_y_list, test_y_list,
 train_denticities_list, test_denticities_list) = train_test_split(atom_features_list,
                                                                   natom_list,
                                                                   edge_index_list,
                                                                   edge_feats_list,
                                                                   y_list,
                                                                   denticities_list,
                                                                   test_size=0.1, shuffle=True)
# split train further into train + val
(train_atom_list, val_atom_list,
 train_natom_list, val_natom_list,
 train_edge_index_list, val_edge_index_list,
 train_edge_feats_list, val_edge_feats_list,
 train_y_list, val_y_list,
 train_denticities_list, val_denticities_list) = train_test_split(train_atom_list,
                                                                  train_natom_list,
                                                                  train_edge_index_list,
                                                                  train_edge_feats_list,
                                                                  train_y_list,
                                                                  train_denticities_list,
                                                                  test_size=1/9, shuffle=True)

In [33]:
class LigandDataset():
    def __init__(self, atom_list, natom_list, edge_index_list, edge_feats_list, y_list, denticities_list):
        self.atom_list = atom_list
        self.natom_list = natom_list
        self.edge_index_list = edge_index_list
        self.edge_feats_list = edge_feats_list
        self.y_list = y_list
        self.denticities_list = denticities_list
        
    def __len__(self):
        return len(self.atom_list)
        
    def __getitem__(self, idx):
        return Data(x=torch.Tensor(self.atom_list[idx]),
                    natoms=torch.Tensor([self.natom_list[idx]]),
                    edge_index=torch.Tensor(np.array(self.edge_index_list[idx])),
                    edge_attr=torch.Tensor(self.edge_feats_list[idx]),
                    # y=torch.Tensor(self.y_list[idx]).unsqueeze(1).to(torch.long),
                    denticity=torch.Tensor([self.denticities_list[idx]]),
                    y=torch.nn.functional.one_hot(torch.Tensor(self.y_list[idx]).to(torch.long), num_classes=2) # one-hot
                   )
    
# Normalize QM features
feats_to_normalize = np.concatenate(train_atom_list)[:, -4:]
maxes = np.max(feats_to_normalize, axis=0)
mins = np.min(feats_to_normalize, axis=0)

denom = (maxes-mins)
for idx in range(len(train_atom_list)):
    train_atom_list[idx][:,-4:] = 2*(train_atom_list[idx][:,-4:] - mins)/denom - 1
for idx in range(len(test_atom_list)):
    test_atom_list[idx][:,-4:] = 2*(test_atom_list[idx][:,-4:] - mins)/denom - 1
for idx in range(len(val_atom_list)):
    val_atom_list[idx][:,-4:] = 2*(val_atom_list[idx][:,-4:] - mins)/denom - 1

train_data = LigandDataset(train_atom_list, train_natom_list, train_edge_index_list,
                           train_edge_feats_list, train_y_list, train_denticities_list)
test_data = LigandDataset(test_atom_list, test_natom_list, test_edge_index_list,
                          test_edge_feats_list, test_y_list, test_denticities_list)
val_data = LigandDataset(val_atom_list, val_natom_list, val_edge_index_list,
                         val_edge_feats_list, val_y_list, val_denticities_list)

In [34]:
np.savez('data/train_qm_normalization_params.npz', maxes=maxes, mins=mins, keys=['partial_charge',
                                                                            'fukui_neu',
                                                                            'fukui_elec',
                                                                            'NMR'], desc='Last four values of Nx18 atom_features')

In [35]:
torch.save(train_data, 'data/train_dataset.pt')
torch.save(test_data, 'data/test_dataset.pt')
torch.save(val_data, 'data/val_dataset.pt')

In [7]:
test = filtered_mol_ls[0]
handler = ReactivityDescriptorHandler()
results = handler.predict(Chem.MolToSmiles(test))
results

{'smiles': 'CCC1=C(CC)c2cc3[n-]c(cc4nc(cc5[n-]c(cc1n2)c(CC)c5CC)C(CC)=C4CC)c(CC)c3CC',
 'partial_charge': array([ 0.03270729,  0.03327774, -0.01462392, -0.01462392,  0.03327774,
         0.03270729,  0.03628747, -0.01064983,  0.03729194, -0.2112923 ,
         0.03729193, -0.01064983,  0.03628748, -0.14713392,  0.03628747,
        -0.01064983,  0.03729193, -0.21129227,  0.03729194, -0.01064984,
         0.03628748, -0.14713389, -0.00504875,  0.03791487,  0.0320563 ,
        -0.00504874,  0.03791486,  0.03205629, -0.01462392,  0.03327773,
         0.03270729, -0.01462394,  0.03327774,  0.03270728, -0.00504874,
         0.03791486,  0.03205629, -0.00504874,  0.03791487,  0.03205629],
       dtype=float32),
 'fukui_neu': array([ 0.00393121,  0.00031168,  0.06010206,  0.06010206,  0.00031168,
         0.00393121,  0.0274829 ,  0.02088268,  0.01483827,  0.11349286,
         0.01483827,  0.02088268,  0.0274829 ,  0.04035588,  0.0274829 ,
         0.02088268,  0.01483827,  0.11349288,  0.01483